## VGG-16 모델로 합성곱 신경망 구현 하기

- 심화버젼

VGG-16 은 옥스포드 대학 연구팀에 의해 개발된 모델(VGGNet)로 2014년 이미지넷 이미지 인식대회(ILSVRC)에서 준우승을 했다. 

이전 모델은 대부분 얕은 층으로 구성된 네트워크를 사용했다면 VGG-16 이후로 층수가 높아지고 있다. 우승을 한 GoogleNet 보다 VGG-16 이 인기가 있는 이유는 간편함 때문이다. 

학습과 속도 관계로 VGG-16 모델을 줄여서 MNIST 학습을 진행해 본다.

Colab 의 [런타임] - [런타임 유형 변경] 을 클릭해서 [GPU]를 사용하도록 설정한다. 학습속도가 빨라진다.

Keras 의 모델 서브클래싱 API 를 이용한다.

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D


# 데이터셋 준비 : MNIST

MNIST 를 가져온다. tensorflow 에서 기본으로 제공하는 데이터셋이므로 mnist.load_data() 를 사용한다.

In [ ]:
# TODO





Convolution Layer 를 사용해야 하므로 28x28 짜리 행렬이 아닌 채널이 추가된 텐서로 수정한다. 이번에는 tf.newaxis 를 사용한다. reshape() 을 사용해도 상관없다.


In [ ]:
# TODO




In [ ]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

shuffle() 을 사용해서 데이터셋을 섞은 후 배치를 만든다.

In [ ]:
# TODO




# VGG-16 의 합성곱 층(Convolution Layer) 만들기

Keras 의 subclassing API 를 사용하여 tf.keras 모델을 만든다.

Conv2D 와 MaxPooling2D 를 사용해서 합성층곱을 정의한다. 

CNN 은 배치크기를 제외하고 이미지의 높이, 너비, 채널 크기를 텐서입력으로 받는다. MNIST 는 흑백이므로 (28,28,1) 크기의 CNN 을 정의해서 사용한다. 

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    # TODO





  def call(self, x):
    # TODO


    return x

model = MyModel()

Conv2D 와 MaxPooling2D 출력은 (높이, 너비, 채널)의 텐서다. Conv2D 를 써서 채널을 늘리고, MaxPooling2D 를 써서 높이와 너비를 줄인다. 계산비용면에서 출력 채널을 늘리기 위해서는 높이와 너비를 줄여야한다.

옵티마이저와 손실함수

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


손실과 성능 측정 지표 선택

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


tf.GradientTape 으로 모델 훈련

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.1363498419523239, 정확도: 95.7316665649414, 테스트 손실: 0.06965141743421555, 테스트 정확도: 97.70999908447266
에포크: 2, 손실: 0.08989054709672928, 정확도: 97.17582702636719, 테스트 손실: 0.051520440727472305, 테스트 정확도: 98.29000091552734
에포크: 3, 손실: 0.0708622857928276, 정확도: 97.77667236328125, 테스트 손실: 0.04616954177618027, 테스트 정확도: 98.48999786376953
에포크: 4, 손실: 0.05919172242283821, 정확도: 98.14250183105469, 테스트 손실: 0.04268050193786621, 테스트 정확도: 98.61000061035156
에포크: 5, 손실: 0.051203809678554535, 정확도: 98.39067077636719, 테스트 손실: 0.039049182087183, 테스트 정확도: 98.73200225830078


# 전체 연결 Dense Layer

모델의 마지막은 Convolution Layer 의 출력 텐서를 2개의 전연결층(Dense)에 연결하고 분류를 진행한다. 

우선 Flatten() 을 사용해서 텐서를 벡터로 변환한다. 그 후 2개의 Dense() 층을 연결한다. 마지막은 softmax 를 사용해서 10개의 분류에 할당되도록 한다.

7x7x64 텐서가 3136 벤터로 Flatten 되었다.

# 모델 컴파일

가장 보편적인 'adam' 과 크로스엔트로피, accuracy 를 사용해서 모델을 컴파일한다.

# 훈련

# 평가

간단한 CNN 모델로 99%의 정확도를 얻었다.